In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("arraypassing.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [2]:
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

hdmi_in.start()
hdmi_out.start()

In [ ]:
hdmi_in.tie(hdmi_out)

In [ ]:
from pynq import MMIO
#filter_reference = MMIO(0x00000000,0x10000)
filter_reference = MMIO(0x83C20000,0x10000)  #MMIO Addresses should always be double-checked when exporting a Vivado Design

In [ ]:
import cv2

for i in range(1000):
    in_frame = hdmi_in.readframe()
    out_frame = hdmi_out.newframe()
    
    filter_reference.write(0x10, in_frame.physical_address)   # Get Pointers to memory
    filter_reference.write(0x18, out_frame.physical_address)
    
    
    
    filter_reference.write(0x20, 1920)   # Make sure that the input HDMI signal is set to 1280x720
    filter_reference.write(0x28, 1080)
    #filter_reference.write(0x20, in_frame.shape[1]) 
    #filter_reference.write(0x28, in_frame.shape[0])
    filter_reference.write(0x00, 0x01)             # ap_start triggering
    while (filter_reference.read(0) & 0x4) == 0:   # ap_done checking
        pass
    #print(out_frame)
    cv2.rectangle(in_frame, (ElmoX-ElmoW,ElmoY), (ElmoX,ElmoY), (255,0,0), 2)
    hdmi_out.writeframe(out_frame)

# NEW CODE  

In [ ]:
import cv2
import random
import time

imgCooki = cv2.imread('Cookie.png', -1)   #Loading Images
imgCookie - cv2.cvtColor(imgCooki, cv2.COLOR_BGR2RGBA)
imgHear = cv2.imread("heart.png", -1)
imgHeart - cv2.cvtColor(imgHear, cv2.COLOR_BGR2RGBA)
imgBro = cv2.imread('Broccoli.png', -1)
imgBroc - cv2.cvtColor(imgBro, cv2.COLOR_BGR2RGBA)


frame_count = 0   #used to create retro looking speed

cookie_point_across_one = random.randint(50, 1200)   #spawn in objects
cookie_point_across_two = random.randint(50,1200)
cookie_point_across_three = random.randint(50,1200)
broc_point_across_one = random.randint(50,1200)
broc_point_across_two = random.randint(50,1200)

        
allgood = True
        
while(allgood):
    if abs(cookie_point_across_two-cookie_point_across_one) < 75:   #checking no collisions between cookies
        cookie_point_across_two = random.randint(50,1200)
    if abs(cookie_point_across_one-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1200)
    elif abs(cookie_point_across_two-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1200)
    else:
        allgood = False
            
allgood = True

while(allgood):   #checking no collisions between cookies and broc
    if abs(broc_point_across_one-cookie_point_across_one) < 75 or abs(broc_point_across_one-cookie_point_across_two) < 75 or abs(broc_point_across_one-cookie_point_across_three) < 75:
        broc_point_across_one = random.randint(50,1200)
    if abs(broc_point_across_two-cookie_point_across_one) < 75 or abs(broc_point_across_two-cookie_point_across_two) < 75 or abs(broc_point_across_two-cookie_point_across_three) < 75:
        broc_point_across_two = random.randint(50,1200)
    elif abs(broc_point_across_two-broc_point_across_one) < 75:
        broc_point_across_two = random.randint(50,1200)
    else:
        allgood = False
            
cookie_point_down_one = 0    #spawning cookies at the top
cookie_point_down_two = 0   
cookie_point_down_three = 0
broc_point_down_one = 0
broc_point_down_two = 0
#lower is faster
cookie_speed = 6    
lives_one = 3
lives_two = 3
score_one = 0
score_two = 0

player_one_x = 70
player_one_y = 600         #these will change with inputs from the cpp code
player_one_length = 150

player_two_x = 500
player_two_y = 600
player_two_length = 150

for _ in range(1000):
    
    frame = hdmi_in.readframe()
    #takes camera in
    
    #setting up the two players
     
    player_one_x_end =  player_one_x+player_one_length       
    player_two_x_end =  player_two_x+player_two_length
    
    player_one_middle = player_one_x+(0.5*player_one_length)
    player_two_middle = player_two_x+(0.5*player_two_length)
         
    cv2.line(frame,(player_one_x, player_one_y), (player_one_x_end, player_one_y), (255,0,0), 2)    #don't have these in final game
    cv2.line(frame,(player_two_x, player_two_y), (player_two_x_end, player_two_y), (0,0,255), 2)    #will have puppets, just for testing
    
    #Setting up variables for dimensions of objects
    
    cookie_h, cookie_w, cookie_c = imgCookie.shape
    broc_h, broc_w, broc_c = imgBroc.shape
    
    broc_bottom_one = broc_point_down_one + broc_h
    broc_bottom_two = broc_point_down_two + broc_h
    
    broc_left_one = broc_point_across_one + broc_w
    broc_left_two = broc_point_across_two + broc_w
    
    cookie_bottom_one = cookie_h + cookie_point_down_one
    cookie_bottom_two = cookie_h + cookie_point_down_two
    cookie_bottom_three = cookie_h + cookie_point_down_three
    
    cookie_left_one = cookie_point_across_one + cookie_w
    cookie_left_two = cookie_point_across_two + cookie_w
    cookie_left_three = cookie_point_across_three + cookie_w
    
    cookie_middle_y_one = cookie_bottom_one-(0.5*cookie_h)
    cookie_middle_y_two = cookie_bottom_two-(0.5*cookie_h)
    cookie_middle_y_three = cookie_bottom_three-(0.5*cookie_h)
    
    cookie_middle_x_one = cookie_point_across_one+(0.5*cookie_w)
    cookie_middle_x_two = cookie_point_across_two+(0.5*cookie_w)
    cookie_middle_x_three = cookie_point_across_three+(0.5*cookie_w)
    
    #Setting up Collisions with player_one
    
    broc_coll_one = False
    broc_coll_two = False

    if (cookie_point_across_one) >= (player_one_x-40) and (cookie_left_one) <= (player_one_x_end+40) and abs(cookie_bottom_one-player_one_y) <= 15 :
        score_one = score_one + 10
        cookie_point_down_one = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)

    if (cookie_point_across_two) >= (player_one_x-40) and (cookie_left_two) <= (player_one_x_end+40) and abs(cookie_bottom_two-player_one_y) <= 15 :
        score_one = score_one + 10
        cookie_point_down_two = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_two = random.randint(50,1200)
                    
    if (cookie_point_across_three) >= (player_one_x-40) and (cookie_left_three) <= (player_one_x_end+40) and abs(cookie_bottom_three-player_one_y) <= 15 :
        score_one = score_one + 10
        cookie_point_down_three = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_threed = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
                    
    if (broc_point_across_one) >= (player_one_x-40) and (broc_left_one) <= (player_one_x_end+40) and abs(broc_bottom_one-player_one_y) <= 15 :
        lives_one = lives_one - 1
        broc_point_down_one = 0
        allgood = True
        broc_coll_one = True
        while(allgood):
            broc_point_across_one = random.randint(50,1200)
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
                
    if (broc_point_across_two) >= (player_one_x-40) and (broc_left_two) <= (player_one_x_end+40) and abs(broc_bottom_two-player_one_y) <= 15 :
        lives_one = lives_one - 1
        broc_point_down_two = 0
        broc_coll_two = True
        allgood = True
        while(allgood):
            broc_point_across_two = random.randint(50,1200)
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
                
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)

    #Setting up Collisions with player_two
    
    broc_coll_one = False
    broc_coll_two = False

    if (cookie_point_across_one) >= (player_two_x-30) and (cookie_left_two) <= (player_two_x_end+30) and abs(cookie_bottom_two-player_one_y) <= 15 :
        score_two = score_two + 10
        cookie_point_down_one = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)

    if (cookie_point_across_two) >= (player_two_x-30) and (cookie_left_two) <= (player_two_x_end+30) and abs(cookie_bottom_two-player_two_y) <= 15 :
        score_two = score_two + 10
        cookie_point_down_two = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_two = random.randint(50,1200)
                    
    if (cookie_point_across_three) >= (player_two_x-30) and (cookie_left_three) <= (player_two_x_end+30) and abs(cookie_bottom_three-player_two_y) <= 15 :
        score_two = score_two + 10
        cookie_point_down_three = 0
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_threed = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
                    
    if (broc_point_across_one) >= (player_two_x-30) and (broc_left_one) <= (player_two_x_end+30) and abs(broc_bottom_one-player_two_y) <= 15 :
        lives_two = lives_two - 1
        broc_point_down_one = 0
        allgood = True
        broc_coll_one = True
        while(allgood):
            broc_point_across_one = random.randint(50,1200)
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
                
    if (broc_point_across_two) >= (player_two_x-30) and (broc_left_two) <= (player_two_x_end+30) and abs(broc_bottom_two-player_two_y) <= 15 :
        lives_two = lives_two - 1
        broc_point_down_two = 0
        allgood = True
        broc_coll_two = True
        while(allgood):
            broc_point_across_two = random.randint(50,1200)
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
                
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)

            
    #Setting up speed
        
    if score_one%50 == 0 or score_two%50 == 0:
            if cookie_speed > 2 and score_one > 0 and score_two > 0:
                cookie_speed = cookie_speed - 1
    
    if frame_count%cookie_speed == 0:
        cookie_point_down_one = cookie_point_down_one+20
        cookie_point_down_two = cookie_point_down_two+20
        cookie_point_down_three = cookie_point_down_three+20
        broc_point_down_one = broc_point_down_one+20
        broc_point_down_two = broc_point_down_two+20

        
    #Setting up bottom collision
    
    broc_coll_one = False
    broc_coll_two = False

        
    if cookie_bottom_one >= 650:
        distance_one = ((player_one_y-cookie_middle_y_one)**2)+((player_one_middle-cookie_middle_x_one)**2)
        distance_two = ((player_two_y-cookie_middle_y_one)**2)+((player_two_middle-cookie_middle_x_one)**2)
        if distance_one<distance_two:
            lives_one = lives_one-1
        if distance_one>distance_two:
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_one = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_one-cookie_point_across_two) > 75 and abs(cookie_point_across_one-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_one = random.randint(50,1200)
            cookie_point_down_one = 0
    
    if cookie_bottom_two >= 650:
        distance_one = ((player_one_y-cookie_middle_y_two)**2)+((player_one_middle-cookie_middle_x_two)**2)
        distance_two = ((player_two_y-cookie_middle_y_two)**2)+((player_two_middle-cookie_middle_x_two)**2)
        if distance_one<distance_two:
            lives_one = lives_one-1
        if distance_one>distance_two:
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_two = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_two-cookie_point_across_one) > 75 and abs(cookie_point_across_two-cookie_point_across_three) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
            cookie_point_down_two = 0
            
    if cookie_bottom_three >= 650:
        distance_one = ((player_one_y-cookie_middle_y_three)**2)+((player_one_middle-cookie_middle_x_three)**2)
        distance_two = ((player_two_y-cookie_middle_y_three)**2)+((player_two_middle-cookie_middle_x_three)**2)
        if distance_one<distance_two:
            lives_one = lives_one-1
        if distance_one>distance_two:
            lives_two = lives_two -1
        if lives_one > 0 and lives_two > 0:
            cookie_point_across_three = random.randint(50,1200)
            allgood = True
            while(allgood):
                if abs(cookie_point_across_three-cookie_point_across_one) > 75 and abs(cookie_point_across_three-cookie_point_across_two) > 75:
                    allgood = False
                else:
                    cookie_point_across_three = random.randint(50,1200)
            cookie_point_down_three = 0
            
    if broc_bottom_one > 650:
        allgood = True
        broc_coll_one = True
        broc_point_across_one = random.randint(50,1770)
        while(allgood):
            if abs(broc_point_across_one-cookie_point_across_one) > 75 and abs(broc_point_across_one-cookie_point_across_two) > 75 and abs(broc_point_across_one-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_one = random.randint(50,1200)
        broc_point_down_one = 0
        
    if broc_bottom_two > 650:
        allgood = True
        broc_coll_two = True
        broc_point_across_two = random.randint(50,1770)
        while(allgood):
            if abs(broc_point_across_two-cookie_point_across_one) > 75 and abs(broc_point_across_two-cookie_point_across_two) > 75 and abs(broc_point_across_two-cookie_point_across_three) > 75:
                allgood = False
            else:
                broc_point_across_two = random.randint(50,1200)
        broc_point_down_two = 0
        
    if broc_coll_one:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_one = random.randint(50,1200)
            
    if broc_coll_two:
        if abs(broc_point_across_two-broc_point_across_one) < 75:
            broc_point_across_two = random.randint(50,1200)


        
    
    heart_h, heart_w, heart_c = imgHeart.shape
    
    #Printing the hearts
    
        
    
    frame_h, frame_w, frame_c = frame.shape
    
    cv2.rectangle(frame, (1280,0), (1130,50), (0,0,0), -1)
    cv2.rectangle(frame, (1280,70), (1130,120), (0,0,0), -1)
    
    y_off=0
    if lives_one == 3:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart       
        x_off=1130
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart
                
                
    if lives_one == 2:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
            
     
    if lives_one == 1:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
    y_off=70
    if lives_two == 3:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart       
        x_off=1130
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart
                
                
    if lives_two == 2:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart   
        x_off=1180
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
        
            
     
    if lives_two == 1:
        x_off=1230
        frame[y_off:y_off+imgHeart.shape[0], x_off:x_off+imgHeart.shape[1]] = imgHeart 
    
            
        
    #Printing the cookies and broc
    
    if lives_one > 0 and lives_two > 0:
        x_off = cookie_point_across_one
        y_off = cookie_point_down_one
        frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie          
        x_off = cookie_point_across_two
        y_off = cookie_point_down_two
        frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie
        x_off = cookie_point_across_three
        y_off = cookie_point_down_three
        frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie
                
        x_off = broc_point_across_one
        y_off = broc_point_down_one
        frame[y_off:y_off+imgBroc.shape[0], x_off:x_off+imgBroc.shape[1]] = imgBroc
        x_off = broc_point_across_two
        y_off = broc_point_down_two
        frame[y_off:y_off+imgBroc.shape[0], x_off:x_off+imgBroc.shape[1]] = imgBroc
        
        

    
    while (lives_one <= 0):
        frame = hdmi_in.readframe()
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(frame, 'GAME OVER - PLAYER TWO WINS', (70, 400), font, 3, (255,255,255), 2, cv2.LINE_AA)
        hdmi_out.writeframe(frame)
        time.sleep(2)
        print("Good")
        break


        
    while (lives_two <= 0):

        frame = hdmi_in.readframe()
        font = cv2.FONT_HERSHEY_TRIPLEX
        cv2.putText(frame, 'GAME OVER - PLAYER ONE WINS', (70, 400), font, 3, (255,255,255), 2, cv2.LINE_AA)
        hdmi_out.writeframe(frame)
        time.sleep(2)
        print("Good")
        break


          
    cv2.putText(frame, str(score_one), (70,50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(score_two), (70,100), cv2.FONT_HERSHEY_TRIPLEX, 2, (0,0,0), 1, cv2.LINE_AA)   
    
    cv2.putText(frame, "PLAYER 1 - COOKIE MONSTER", (400,50), cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "PLAYER 2 - ELMO", (400,100), cv2.FONT_HERSHEY_TRIPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
        

    
    frame_count = frame_count + 1
    hdmi_out.writeframe(frame)
    





In [ ]:
import cv2
#import numpy as np
import random



imgCookie = cv2.imread('Cookie.png', -1)
#imgCookie = cv2.cvtColor(imgCookie, cv2.COLOR_BGRA2RGBA)
imgHeart = cv2.imread("Cookie.png", -1)
#imgHeart = cv2.cvtColor(imgHeart, cv2.COLOR_BGRA2RGBA)
imgBroc = cv2.imread('Cookie.png', -1)
#imgBroc = cv2.cvtColor(imgBroc, cv2.COLOR_BGRA2RGBA)


frame_count = 0   #used to create retro looking speed

cookie_point_across_one = random.randint(50, 1770)   #spawn in objects
cookie_point_across_two = random.randint(50,1770)
cookie_point_across_three = random.randint(50,1770)
broc_point_across_one = random.randint(50,1770)
broc_point_across_two = random.randint(50,1770)

        
allgood = True
        
while(allgood):
    if abs(cookie_point_across_two-cookie_point_across_one) < 75:   #checking no collisions between cookies
        cookie_point_across_two = random.randint(50,1770)
    if abs(cookie_point_across_one-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1770)
    elif abs(cookie_point_across_two-cookie_point_across_three) < 75:
        cookie_point_across_three = random.randint(50,1770)
    else:
        allgood = False
            
allgood = True

while(allgood):   #checking no collisions between cookies and broc
    if abs(broc_point_across_one-cookie_point_across_one) < 75 or abs(broc_point_across_one-cookie_point_across_two) < 75 or abs(broc_point_across_one-cookie_point_across_three) < 75:
        broc_point_across_one = random.randint(50,1770)
    if abs(broc_point_across_two-cookie_point_across_one) < 75 or abs(broc_point_across_two-cookie_point_across_two) < 75 or abs(broc_point_across_two-cookie_point_across_three) < 75:
        broc_point_across_two = random.randint(50,1770)
    elif abs(broc_point_across_two-broc_point_across_one) < 75:
        broc_point_across_two = random.randint(50,1770)
    else:
        allgood = False
            
cookie_point_down_one = 0    #spawning cookies at the top
cookie_point_down_two = 0   
cookie_point_down_three = 0
broc_point_down_one = 0
broc_point_down_two = 0
#lower is faster
cookie_speed = 6    
lives_one = 3
lives_two = 3
score_one = 0
score_two = 0

player_one_x = 70
player_one_y = 600         #these will change with inputs from the cpp code
player_one_length = 300

player_two_x = 1300
player_two_y = 600
player_two_length = 300



import time

numframes = 100
start = time.time()

lives_one = 3

for i in range(numframes):
    frame = hdmi_in.readframe()


     
    cookie_h, cookie_w, cookie_c = imgCookie.shape
    broc_h, broc_w, broc_c = imgBroc.shape

        
    yoff = 50
    xoff = 600
    
    if lives_one > 0:
            if cookie
    frame[yoff:yoff+imgCookie.shape[0], xoff:xoff+imgCookie.shape[1]] = imgCookie
    
    
    
   # if lives_one > 0 and lives_two > 0:
   #     x_off = cookie_point_across_one
   #     y_off = cookie_point_down_one
    #    frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie          
   #     x_off = cookie_point_across_two
   #     y_off = cookie_point_down_two
   #     frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie
   #     x_off = cookie_point_across_three
   #     y_off = cookie_point_down_three
   #     print("cookie three: ", x_off)            
   #     print("cookie three: ", y_off) 
   #     frame[y_off:y_off+imgCookie.shape[0], x_off:x_off+imgCookie.shape[1]] = imgCookie
   #     x_off = broc_point_across_one
   #     y_off = broc_point_down_one
   #     print(x_off)            
   #     print(y_off) 
   #     frame[y_off:y_off+imgBroc.shape[0], x_off:x_off+imgBroc.shape[1]] = imgBroc
   #     x_off = broc_point_across_two
   #     y_off = broc_point_down_two
   #     frame[y_off:y_off+imgBroc.shape[0], x_off:x_off+imgBroc.shape[1]] = imgBroc
                
    hdmi_out.writeframe(frame)  

end = time.time()
print(hdmi_out.mode)

print("Frames per second:  " + str(numframes / (end - start)))
    
    




In [ ]:
hdmi_in.close()    # Don't forget to run this to free memory
hdmi_out.close()   # NEVERFORGET NEVERFORGET NEVERFORGET :p 